# Quick example of implementing a simple payments network

There are 4 steps to running Veriphi guardrails on a tool given to an autonomous Agent.
 - Define network classes
 - Prepare data
 - Distribute and encrypt
 - Reconstruct and recover 

In [1]:
import os
import json
import numpy as np

from veriphi_core import utils as utils
from veriphi_core import interface as ic

In [2]:
net_dict = {"Agent": 0, 
            "Authoriser": 1, 
            "Endpoint": 2}

## Prepare data

In [3]:
wallet = {
    "walletId": "abc123",
    "owner": {
        "name": "Jane Doe",
        "id": "user-678",
    },
    "defaultCurrency": "USD",
    "addresses": {
        "BTC": "1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa",
        "ETH": "0x89205A3A3b2A69De6Dbf7f01ED13B2108B2c43e7",
    },
}

# Serialize to JSON string
wallet_json = json.dumps(wallet)

# Convert to UTF-8 bytes (like TextEncoder in TS)
wallet_bytes = wallet_json.encode("utf-8")

In [4]:
wallet_bytes

b'{"walletId": "abc123", "owner": {"name": "Jane Doe", "id": "user-678"}, "defaultCurrency": "USD", "addresses": {"BTC": "1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa", "ETH": "0x89205A3A3b2A69De6Dbf7f01ED13B2108B2c43e7"}}'

In [5]:
len(wallet_bytes)

210

In [6]:
# Generate a public key for distribution and a private key for obfuscation
setupNode = ic.SetupNode('node_setup')
seed = np.frombuffer(os.urandom(32),dtype=np.uint8)
pub_key = setupNode.gen_public_key(seed)
priv_key = setupNode.gen_private_key('obf_private_key',seed)
encrypted, nonce = setupNode.encrypt_data(wallet_bytes, priv_key)
low_val, high_val = setupNode.implement_conditions(0.0,1000.0,priv_key)
encrypted, chunk_size, padding = setupNode.obfuscate_data(np.frombuffer(encrypted,dtype=np.uint8), priv_key, low_val, high_val, 50.0)

### The above cell can be wrapped up in a single function call

In [7]:
public_data, private_data = ic.setup_node(wallet_bytes,0,1000,True)

In [8]:
# pack the data for transmission
agent_packet    = setupNode.package_data(encrypted, pub_key,'E3',net_dict["Agent"])
auth_packet     = setupNode.package_data(encrypted, pub_key,'E3',net_dict["Authoriser"])
endpoint_packet = setupNode.package_data(encrypted, pub_key,'E3',net_dict["Endpoint"])

Setup package identity reported as 0
Setup package identity reported as 1
Setup package identity reported as 2


### More generally for many parties

In [9]:
agent_packet, auth_packet, endpoint_packet = ic.distribute_data(public_data,'E',3)

Setup package identity reported as 0
Setup package identity reported as 1
Setup package identity reported as 2


## Distribute and encrypt

### Example code for Agent members

In [10]:
agent_node = ic.EncryptNode('agent_member')
public_key, packet_data, mode, identity = agent_node.unpackage_data(agent_packet)
agent_priv_key = agent_node.gen_private_key('private_key', np.frombuffer(os.urandom(32),dtype=np.uint8))
agent_encrypted = agent_node.encrypt_data(packet_data, agent_priv_key, public_key, mode, identity)
agent_enc_packet = agent_node.package_data(agent_encrypted, mode, identity)

# Or with simple function calls

In [11]:
agent_enc_packet = ic.encrypt_node(agent_packet,"agent_member")
auth_enc_packet  = ic.encrypt_node(auth_packet,"authoriser_member")
endp_enc_packet  = ic.encrypt_node(endpoint_packet,"endpoint_member")

## Reconstruction and recovery is conditioned on complete visibility and correct conditions

In [12]:
veriphier = ic.DecryptNode('Veriphier')
party_data = veriphier.collect_packets(agent_enc_packet, auth_enc_packet, endp_enc_packet)
party_data_recov = veriphier.recover_packets(party_data)
stream_list = veriphier.reconstruct_data(party_data_recov)
reconstructed = veriphier.reassemble_data(stream_list, party_data[0]['mode'])
recovered, chunk_size = veriphier.obfuscate_data(reconstructed, private_data["key"], private_data["low_val"], private_data["high_val"], 500)
recovered = recovered[:-private_data["padding"]]
decrypted = veriphier.decrypt_data(recovered.tobytes(), private_data["nonce"], private_data["key"])

In [13]:
chunk_size

5

In [14]:
recovered

array([], dtype=uint8)

### Or wrapped up in a single call

In [15]:
decrypted = ic.decrypt_node(private_data, 500, True, agent_enc_packet, auth_enc_packet, endp_enc_packet)

### Now we can check the output!

In [16]:
decrypted.decode("utf-8")

'{"walletId": "abc123", "owner": {"name": "Jane Doe", "id": "user-678"}, "defaultCurrency": "USD", "addresses": {"BTC": "1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa", "ETH": "0x89205A3A3b2A69De6Dbf7f01ED13B2108B2c43e7"}}'